
# Importing the necessary libraries 

In [14]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
# import numpy as np
# from sklearn.model_selection import train_test_split
# import pandas as pd
# from keras.models import Sequential
# import tensorflow as tf
# import os
# import cv2
# import matplotlib.pyplot as plt
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.optimizers import RMSprop
# from PIL import Image

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
import cv2
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Sequential, Model,load_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D,Dropout
import tensorflow as tf
import splitfolders 
import pandas as pd
import glob
from sklearn.metrics import confusion_matrix
import itertools
import plotly.graph_objects as go
import plotly.express as px
#Suppressing Warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
! pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import opendatasets as od

In [4]:
data_url='https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection'
### data_url='https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri'

### dataset containing the brain MRIs

In [5]:
od.download(data_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aswinramanathan
Your Kaggle Key: ··········


100%|██████████| 15.1M/15.1M [00:00<00:00, 185MB/s]

In [10]:
#data_dir='/brain-mri-images-for-brain-tumor-detection'
data_dir='/content/brain-mri-images-for-brain-tumor-detection'

In [11]:
!ls -lh {data_dir}

total 12K
drwxr-xr-x 4 root root 4.0K Aug  3 11:06 brain_tumor_dataset
drwxr-xr-x 2 root root 4.0K Aug  3 11:06 no
drwxr-xr-x 2 root root 4.0K Aug  3 11:06 yes


# Splits the images into train, test and validation sets automatically

In [33]:
splitfolders.ratio(data_dir, output="output", seed=101, ratio=(.8, .1, .1))

Copying files: 253 files [00:00, 4475.99 files/s]


In [17]:
train_path='/content/output/train'
test_path='/content/output/test'
val_path='/content/output/val'

In [19]:
from keras.preprocessing.image import ImageDataGenerator

In [40]:

train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(train_path,target_size=(224, 224),batch_size=32,shuffle=True,class_mode='binary',classes=['yes','no'])
test_generator = test_datagen.flow_from_directory(test_path,target_size=(224,224),batch_size=32,shuffle=False,class_mode='binary',classes=['yes','no'])
val_generator = val_datagen.flow_from_directory(val_path,target_size=(224,224),batch_size=32,shuffle=False,class_mode='binary',classes=['yes','no'])

Found 202 images belonging to 2 classes.
Found 27 images belonging to 2 classes.
Found 24 images belonging to 2 classes.


# Calling ResNet


In [21]:
from tensorflow.keras.applications import ResNet50

In [22]:
model = ResNet50(
      input_shape = (224,224,3),
      include_top = False,
      weights = 'imagenet'
    )
for layers in model.layers:
    layers.trainable = False

94781440/94765736 [==============================] - 0s 0us/step


In [43]:
from keras.layers import  Dropout
x = Flatten()(model.output)
x = Dropout(0.5)(x)
x = Dense(1, activation = "sigmoid")(x)

model = keras.Model(model.input, x)
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [36]:
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=15)

In [44]:
mc = ModelCheckpoint('./output/model.h5', monitor='val_accuracy', mode='max' )
H = model.fit_generator(train_generator,validation_data=val_generator,epochs=150,verbose=2, callbacks=[es,mc])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/150
7/7 - 8s - loss: 0.8548 - accuracy: 0.4851 - val_loss: 0.8015 - val_accuracy: 0.3750 - 8s/epoch - 1s/step
Epoch 2/150
7/7 - 3s - loss: 0.8060 - accuracy: 0.5248 - val_loss: 0.7982 - val_accuracy: 0.3750 - 3s/epoch - 491ms/step
Epoch 3/150
7/7 - 3s - loss: 0.8279 - accuracy: 0.5248 - val_loss: 0.7948 - val_accuracy: 0.3750 - 3s/epoch - 487ms/step
Epoch 4/150
7/7 - 3s - loss: 0.8356 - accuracy: 0.4950 - val_loss: 0.7915 - val_accuracy: 0.3750 - 3s/epoch - 486ms/step
Epoch 5/150
7/7 - 3s - loss: 0.8034 - accuracy: 0.5198 - val_loss: 0.7885 - val_accuracy: 0.3750 - 3s/epoch - 484ms/step
Epoch 6/150
7/7 - 3s - loss: 0.7900 - accuracy: 0.5248 - val_loss: 0.7854 - val_accuracy: 0.3750 - 3s/epoch - 480ms/step
Epoch 7/150
7/7 - 3s - loss: 0.8419 - accuracy: 0.4802 - val_loss: 0.7824 - val_accuracy: 0.3750 - 3s/epoch - 486ms/step
Epoch 8/150
7/7 - 3s - loss: 0.7966 - accuracy: 0.5248 - val_loss: 0.7795 - val_accuracy: 0.3750 - 3s/epoch - 484ms/step
Epoch 9/150
7/7 - 3s - loss: 0.7902

# Plotting the validation accuracy and model accuracy

In [ ]:
history = model1.fit(train_x, train_y,validation_split = 0.1, epochs=50, batch_size=4)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Plotting the validation loss and training loss

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Visualizing the actual model

In [ ]:
###    https://towardsdatascience.com/visualizing-keras-models-4d0063c8805e
### to visualise the layers

In [ ]:
### https://github.com/https-deeplearning-ai/tensorflow-1-public/blob/main/C2/W1/ungraded_lab/C2_W1_Lab_1_cats_vs_dogs.ipynb
### to visualize intermediate images in a CNN

# Ideas to try out


### try confusion matrix
### try pre trained models
### try augmenting    https://www.kaggle.com/code/ruslankl/brain-tumor-detection-v1-0-cnn-vgg-16